In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!which python

/links/groups/hierlemann/spikesorting/miniconda3/envs/si2/bin/python


In [3]:
import spikeinterface.full as si
import spikeinterface.sortingcomponents as sc
import belextractors as be

from pathlib import Path
import matplotlib.pyplot as plt

import h5py
import numpy as np

import time

%matplotlib widget

In [4]:
# This is to check the h5 files inside the recording folder
ncmos_folder = Path("/net/bs-filesvr02/export/group/hierlemann/intermediate_data/NeuroCMOS/aforastiere/Thesis/20220531/Chip236/scan01")
files_list = [p for p in ncmos_folder.iterdir() if "h5" in p.suffix]

print(files_list)

FileNotFoundError: [Errno 2] No such file or directory: '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/NeuroCMOS/aforastiere/Thesis/20220531/Chip236/scan01'

In [4]:
#Brain Slice Experiment 20220502_p1p1_scan01
ncmos_fp = "/net/bs-filesvr02/export/group/hierlemann/intermediate_data/NeuroCMOS/aforastiere/Thesis/First_neuroplating/20220531-Hasan/Chip148/scan01/ncmos_20220531_133019.h5"


In [5]:
f = h5py.File(ncmos_fp)
rec_names = list(f['ephys'].keys())

electrode_rates = {}
all_peaks = []
all_rec_fs = []

for i, rec_name in enumerate(rec_names[:]):
    print('Loading {}/{}...'.format(i+1, len(rec_names)))
    rec = be.NeuroCMOSRecordingExtractor(Path(ncmos_fp), rec_name)
    all_rec_fs.append(si.bandpass_filter(rec))
    duration = rec.get_num_samples() / rec.get_sampling_frequency()
    
    all_peaks.append(sc.detect_peaks(all_rec_fs[-1], n_jobs=8, total_memory="1G", progress_bar=True))
    un_ind, counts = np.unique(all_peaks[-1]["channel_ind"], return_counts=True)
    rates = counts / duration
    electrodes = all_rec_fs[-1].get_property("electrode")
    
    for (ind, rate) in zip(un_ind, rates):
        electrode_rates[electrodes[ind]] = rate
print('Done')

Loading 1/10...


detect peaks: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:11<00:00,  1.76it/s]


Loading 2/10...


detect peaks: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:11<00:00,  1.77it/s]


Loading 3/10...


...

Loading 9/10...


detect peaks: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:11<00:00,  1.75it/s]


Loading 10/10...


detect peaks: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:11<00:00,  1.73it/s]

Done


In [6]:
def get_xy_from_vertices(x):
    return np.mean(x, axis=0)/13.5


X = np.zeros((180, 332))

for peaks, rec_fs in zip(all_peaks, all_rec_fs):
    # First we extract all the x,y positions for each channel
    probe = rec_fs.get_probes()[0]
    all_vertices = probe.get_contact_vertices()
    num_electrodes = len(all_vertices)
    
    positions = np.zeros((num_electrodes, 2))
    for i, vertex in enumerate(all_vertices):
        positions[i,:] = get_xy_from_vertices(vertex)
    
    
    for peak in peaks:
        # Indices 
        # 0: sample, 1: channel, 2: amplitude, 3: we don't care
        peak_electrode_position = positions[peak[1], :]
        X[int(peak_electrode_position[1])-1, int(peak_electrode_position[0])-1] += 1
        
    
    

In [17]:
# Plot the activity map from the scan
from scipy.ndimage import gaussian_filter

fig=plt.figure()
cax=plt.imshow(gaussian_filter(X, sigma=3), cmap='plasma', vmin=0,vmax=1) #You can change vmin to set the noise level
#Hack to arrange the X and Y axis to the array size
a = np.array([0,500,1000,1500,2000,2500,3000,3500,4000])/13.5
b = np.array([0,500,1000,1500,2000])/13.5
my_xticks = np.array([0,500,1000,1500,2000,2500,3000,3500,4000])
my_yticks = np.array([0,500,1000,1500,2000])
plt.xticks(a, my_xticks)
plt.yticks(b, my_yticks)

plt.xlabel("x ($\\mu$m)", fontdict = {'size':14})
plt.ylabel("y ($\\mu$m)", fontdict = {'size':14})


#cbar = fig.colorbar(cax, ticks=[0])
#cbar =plt.colorbar()
cbar = fig.colorbar(cax, ticks=[0,1])
cbar.ax.set_yticklabels(['0','1'])
cbar.set_label('Detected peaks', rotation=90)


plt.show()

#You can use following command to save the image in pdf for better quality
plt.savefig("scan01.png")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])